In [107]:
import librosa
import numpy as np
import sys
import pickle
import time
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy import signal

**Import some audio file**

In [108]:
filename = 'taxmanShort.wav'
x, sr = librosa.load(filename)
n = 85
hop = 700


**Define some filters**

In [109]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band', analog=False)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

**Encode/Decode**
Define encode and decode functions here

In [110]:
#encode function


def encode(a):
    x_lowpass = butter_bandpass_filter(a, 10, sr/3, sr)
    x_sampled = librosa.resample(x_lowpass,sr,2*sr/3)
    D = librosa.stft(x_sampled, hop_length=hop).T
    
   
    encoded = np.zeros((D.shape[0],3,n))
    # q determines the width of the triangles
# q is the ratio of the center frequency to the frequency width
    q = 6.5

    index = 0
    for dft in D:

        # empty lists to store amplitude and frequency values
        A = []
        F = []
        P = []

        #make a copy of the magnitude dft
        length = np.shape(dft)[0]
        magSpec = np.absolute(dft)
        freq = (np.arange(length)/length) * (sr//2)
        phase = np.angle(dft)

        # n times
        nRange = n
        iii = 0
        while iii < nRange:
            
            ########################## To Do ##########################
            # determine the index corresponding to maximum magnitude
            # store in i
            # hint: use np.argmax()
            i = np.argmax(magSpec)

            ###########################################################

            ########################## To Do ##########################
            # store the maximum magnitude value in h
            # (this will be the height of our triangle)
            h = magSpec[i]

            ###########################################################

            ########################## To Do ##########################
            # save the chosen frequency and amplitude in lists A, F
            # hint: use append()
            
            F.append(freq[i])
            P.append(phase[i])
            A.append(h)

            
            ###########################################################

            # compute half win size
            # width is i/q
            # halfWin will be floor(width/2)
            halfWin = int(np.floor(i/(q*2)))

            ########################## To Do ##########################
            # generate the ramp up (first half of triangle)
            # number of points should be size of halfWin
            # don't include the end point
            # hint: use np.linspace()
            rampUp = np.linspace(0,h, num=halfWin,endpoint=False)
            #rampUp = h - (count*8.69/halfWin)
            np.linspace(0,h, num=halfWin,endpoint=False)
            ###########################################################

            ########################## To Do ##########################
            # generate the ramp down
            # hint: use np.flip()
            rampDown = np.flip(rampUp, axis=0)
            ###########################################################

            ########################## To Do ##########################
            # combine your ramp up, the triangle peak h, and the ramp down
            # to get array containing the masking triangle.
            # it should have length (2 * halfWin + 1)
            # hint: use np.concatenate()
            triangle = np.concatenate([rampUp, [h], rampDown])
            ###########################################################

            # the following code takes the triangle and places it in
            # the correct position in an array so that the peak lines
            # up with the correct magSpec index
            zeros = np.zeros(length)
            padded = np.concatenate((zeros,triangle,zeros),axis = 0)
            shift = length + halfWin - i
            tri = padded[shift:(shift + length)]

            # this code updates magSpec according to the rule:
            # add a zero if the magSpec value is under the triangle
            # otherwise, add the magSpec value 
            magSpec = np.array([0 if (magSpec[idx] <= tri[idx]) else magSpec[idx] for idx in range(length)])
            iii+=1

        # store them in a
        encoded[index][0] = A
        encoded[index][1] = F
        encoded[index][2] = P

        #increment index
    
        index += 1
            
    return encoded

In [111]:
#decode function
def decode(a):
    index = 0
    decoded = np.zeros((1,0))
    for frame in a:
        
        A = frame[0]
        F = frame[1]
        P = frame[2]

        dur = hop / sr

        t = np.linspace(0,dur,int(dur*sr))

        F.shape = (n,1)
        A.shape = (1,n)
        P.shape = (n,1)
        t.shape = (1,len(t))

        # Generate matrix of sinusoids
        #sine_mat = np.exp((2j * np.pi * F * t)+P)
        #sine_sum = np.dot(A, sine_mat)
        # Weight by amplitude vector A and sum
        sine_sum = np.dot(A,np.sin(2*np.pi*F*t) + P)

        # append sine sum onto y_out vector
        decoded = np.concatenate((decoded,sine_sum),axis=1)
        
        index = index + 1
    
    #averaging filter
    avgFil = (1/8)*np.ones((8,1))
    
    #attempts at using Pascal's triangle for filters
    #avgFil = np.reshape((1/36)*np.array([1,5,10,10,5,1]),(1,6))
    #avgFil = np.reshape((1/100)*np.array([1,9,36,84,126,126,84,36,9,1]),(1,10))
    
    #power filters?
    #avgFil = np.reshape((1/36)*np.array([1,4,9,16,25,36, 36,25,16,9,4,1]),(1,12))
    decoded = librosa.resample(decoded, 2*sr/3, sr)
    
    decoded = signal.convolve(avgFil, decoded)
    decoded = butter_bandpass_filter(decoded, 20, sr/3, sr)
    decoded = decoded / np.linalg.norm(decoded)
    #decoded = butter_bandpass_filter(decoded, 20, 3000, sr)
    
    
    return decoded

In [112]:
#runtime section


encodeStartTime = time.time()
x_encoded = np.zeros(D.shape)
x_encoded = encode(x)
ert = time.time()-encodeStartTime

decodeStartTime = time.time()
x_decoded = decode(x_encoded)
drt = time.time()-decodeStartTime

**Compression Ratio**

In [113]:
def compressionRatio(original, encoded):
    o_str = pickle.dumps(original)
    e_str = pickle.dumps(encoded)
    return sys.getsizeof(o_str)/sys.getsizeof(e_str)
    
cr = compressionRatio(x, x_encoded)

**SNR**

Compare the original file size to the compressed file size

In [114]:
def signalToNoise(original, decoded):
    #force the signals to be the same length
    diff = len(original) - len(decoded)
    if diff < 0:
        original = np.resize(original, (1,len(decoded)))  
    elif diff > 0:
        decoded = np.resize(decoded, (1,len(original)))
    
    #compute snr
    signal = np.power(abs(original), 2)
    noise = np.power(abs(original) - abs(decoded), 2)
    signal = np.where(signal == 0, np.finfo(np.float32).eps, signal)
    noise = np.where(noise == 0, np.finfo(np.float32).eps, noise)
    
    return np.mean(10*np.log10(signal/noise))

snr = signalToNoise(x, x_decoded)

**Evaluate Codec**

Print out evaluation of codec. Listen to the results.

In [116]:
#119952
print("Compression Ratio: ", cr)
print()
print("Total Runtime: ", str(round(ert+drt)))
print("\tEncode Runtime: ", str(round(ert)))
print("\tDecode Runtime: ", str(round(drt)))
print()
print("SNR: ", round(snr,4))
ipd.Audio(decode(encode(x)), rate=sr)
#plt.specgram(decode(encode(x))[0], Fs=sr)

Compression Ratio:  2.0483773245785684

Total Runtime:  15
	Encode Runtime:  15
	Decode Runtime:  1

SNR:  0.1625
